In [8]:
import os
from datetime import datetime
import snowflake.connector

In [9]:
SNOWFLAKE_ACCOUNT = os.getenv("SNOWFLAKE_ACCOUNT")
SNOWFLAKE_USER = os.getenv("SNOWFLAKE_USER")
SNOWFLAKE_PASSWORD = os.getenv("SNOWFLAKE_PASSWORD")
SNOWFLAKE_WAREHOUSE = os.getenv("SNOWFLAKE_WAREHOUSE")
SNOWFLAKE_DATABASE = os.getenv("SNOWFLAKE_DATABASE")
SNOWFLAKE_ROLE = os.getenv("SNOWFLAKE_ROLE")
SNOWFLAKE_SCHEMA_RAW = os.getenv("SNOWFLAKE_SCHEMA_RAW", "RAW")
SNOWFLAKE_SCHEMA_ANALYTICS = os.getenv("SNOWFLAKE_SCHEMA_ANALYTICS", "ANALYTICS")

print(f"Configuración:")
print(f"  - Database: {SNOWFLAKE_DATABASE}")
print(f"  - Schema RAW: {SNOWFLAKE_SCHEMA_RAW}")
print(f"  - Schema ANALYTICS: {SNOWFLAKE_SCHEMA_ANALYTICS}")


Configuración:
  - Database: NYC_TAXI
  - Schema RAW: RAW
  - Schema ANALYTICS: ANALYTICS


In [ ]:
def get_snowflake_conn():
    return snowflake.connector.connect(
        user=os.getenv("SNOWFLAKE_USER"),
        password=os.getenv("SNOWFLAKE_PASSWORD"),
        account=os.getenv("SNOWFLAKE_ACCOUNT"),
        warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
        database=os.getenv("SNOWFLAKE_DATABASE"),
        schema=os.getenv("SNOWFLAKE_SCHEMA_RAW", "RAW"),
        role=os.getenv("SNOWFLAKE_ROLE"),
        client_session_keep_alive=True,
    )
print("\nProbando conexión a Snowflake...")
conn = get_snowflake_conn()
cur = conn.cursor()
cur.execute("SELECT CURRENT_VERSION()")
version = cur.fetchone()[0]
print(f" Conectado a Snowflake versión: {version}")


Probando conexión a Snowflake...
 Conectado a Snowflake versión: 9.37.0


In [11]:
cur.execute(f"CREATE SCHEMA IF NOT EXISTS {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA_ANALYTICS}")
print(f"Schema {SNOWFLAKE_SCHEMA_ANALYTICS} verificado/creado")

Schema ANALYTICS verificado/creado


In [12]:
print(f"\n{'='*80}")
print(f"CONSTRUYENDO OBT_TRIPS")
print(f"{'='*80}")

create_obt_sql = f"""
CREATE OR REPLACE TABLE {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA_ANALYTICS}.OBT_TRIPSV2 AS
SELECT 
    -- Tiempo
    PICKUP_DATETIME,
    DROPOFF_DATETIME,
    DATE(PICKUP_DATETIME) AS PICKUP_DATE,
    HOUR(PICKUP_DATETIME) AS PICKUP_HOUR,
    DATE(DROPOFF_DATETIME) AS DROPOFF_DATE,
    HOUR(DROPOFF_DATETIME) AS DROPOFF_HOUR,
    DAYOFWEEK(PICKUP_DATETIME) AS DAY_OF_WEEK,
    MONTH(PICKUP_DATETIME) AS MONTH,
    YEAR(PICKUP_DATETIME) AS YEAR,
    
    -- Ubicación
    PU_LOCATION_ID,
    PU_ZONE,
    PU_BOROUGH,
    PU_SERVICE_ZONE,
    DO_LOCATION_ID,
    DO_ZONE,
    DO_BOROUGH,
    DO_SERVICE_ZONE,
    
    -- Servicio y códigos
    SERVICE_TYPE,
    VENDOR_ID,
    VENDOR_NAME,
    RATE_CODE_ID,
    RATE_CODE_DESC,
    PAYMENT_TYPE,
    PAYMENT_TYPE_DESC,
    TRIP_TYPE,
    TRIP_TYPE_DESC,
    
    -- Viaje
    PASSENGER_COUNT,
    TRIP_DISTANCE,
    STORE_AND_FWD_FLAG,
    
    -- Tarifas
    FARE_AMOUNT,
    EXTRA,
    MTA_TAX,
    TIP_AMOUNT,
    TOLLS_AMOUNT,
    IMPROVEMENT_SURCHARGE,
    CONGESTION_SURCHARGE,
    AIRPORT_FEE,
    CBD_CONGESTION_FEE,
    EHAIL_FEE,
    TOTAL_AMOUNT,
    
    -- Derivadas
    ROUND(
        DATEDIFF(SECOND, PICKUP_DATETIME, DROPOFF_DATETIME) / 60.0, 
        2
    ) AS TRIP_DURATION_MIN,
    
    CASE 
        WHEN TRIP_DISTANCE > 0 
         AND DATEDIFF(SECOND, PICKUP_DATETIME, DROPOFF_DATETIME) > 0
        THEN ROUND(
            TRIP_DISTANCE / (DATEDIFF(SECOND, PICKUP_DATETIME, DROPOFF_DATETIME) / 3600.0),
            2
        )
        ELSE NULL
    END AS AVG_SPEED_MPH,
    
    CASE 
        WHEN FARE_AMOUNT > 0 AND TIP_AMOUNT IS NOT NULL
        THEN ROUND((TIP_AMOUNT / FARE_AMOUNT) * 100, 2)
        ELSE 0.0
    END AS TIP_PCT,
    
    -- Lineage/Calidad
    RUN_ID,
    INGESTED_AT_UTC,
    SOURCE_YEAR,
    SOURCE_MONTH

FROM {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA_RAW}.UNIFIED_TRIPSV2
"""

print("Creando OBT_TRIPS (esto puede tomar varios minutos)...")
cur.execute(create_obt_sql)
print("Tabla OBT_TRIPS creada exitosamente")



CONSTRUYENDO OBT_TRIPS
Creando OBT_TRIPS (esto puede tomar varios minutos)...
Tabla OBT_TRIPS creada exitosamente


In [31]:
print("\nConfigurando idempotencia (PRIMARY KEY)...")

try:
    cur.execute(f"""
        ALTER TABLE {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA_ANALYTICS}.OBT_TRIPS
        DROP PRIMARY KEY
    """)
    print("PRIMARY KEY existente eliminada")
except Exception as e:
    print(f"No había PRIMARY KEY previa (normal en primera ejecución)")

try:
    cur.execute(f"""
        ALTER TABLE {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA_ANALYTICS}.OBT_TRIPS
        ADD PRIMARY KEY (PICKUP_DATETIME, DROPOFF_DATETIME, PU_LOCATION_ID, DO_LOCATION_ID, SERVICE_TYPE)
    """)
    print("PRIMARY KEY")
except Exception as e:
    print(f"Error creando PRIMARY KEY: {e}")



Configurando idempotencia (PRIMARY KEY)...
PRIMARY KEY existente eliminada
PRIMARY KEY


In [33]:
def reingest_month_delete_insert(conn, year: int, month: int):
    year = int(year)
    month = int(month)
    with conn.cursor() as c:
        print(f"\n{'='*90}")
        print(f" Reingesta mensual {year}-{month:02d}")
        print(f"{'='*90}")

        before_sql = f"""
        SELECT COUNT(*) 
        FROM {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA_ANALYTICS}.OBT_TRIPS
        WHERE YEAR(PICKUP_DATETIME) = {year} AND MONTH(PICKUP_DATETIME) = {month}
        """
        c.execute(before_sql)
        before_count = c.fetchone()[0]
        print(f"Registros previos en {year}-{month:02d}: {before_count:,}")

        try:
            c.execute("BEGIN")

            delete_sql = f"""
            DELETE FROM {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA_ANALYTICS}.OBT_TRIPS
            WHERE YEAR(PICKUP_DATETIME) = {year} AND MONTH(PICKUP_DATETIME) = {month}
            """
            c.execute(delete_sql)
            deleted = c.rowcount or 0
            print(f"Filas eliminadas: {deleted:,}")

            insert_sql = f"""
            INSERT INTO {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA_ANALYTICS}.OBT_TRIPS (
              PICKUP_DATETIME, DROPOFF_DATETIME, PICKUP_DATE, PICKUP_HOUR, DROPOFF_DATE, DROPOFF_HOUR,
              DAY_OF_WEEK, MONTH, YEAR,
              PU_LOCATION_ID, PU_ZONE, PU_BOROUGH, PU_SERVICE_ZONE,
              DO_LOCATION_ID, DO_ZONE, DO_BOROUGH, DO_SERVICE_ZONE,
              SERVICE_TYPE, VENDOR_ID, VENDOR_NAME, RATE_CODE_ID, RATE_CODE_DESC,
              PAYMENT_TYPE, PAYMENT_TYPE_DESC, TRIP_TYPE, TRIP_TYPE_DESC,
              PASSENGER_COUNT, TRIP_DISTANCE, STORE_AND_FWD_FLAG,
              FARE_AMOUNT, EXTRA, MTA_TAX, TIP_AMOUNT, TOLLS_AMOUNT,
              IMPROVEMENT_SURCHARGE, CONGESTION_SURCHARGE, AIRPORT_FEE,
              CBD_CONGESTION_FEE, EHAIL_FEE, TOTAL_AMOUNT,
              TRIP_DURATION_MIN, AVG_SPEED_MPH, TIP_PCT,
              RUN_ID, INGESTED_AT_UTC, SOURCE_YEAR, SOURCE_MONTH
            )
            SELECT
              PICKUP_DATETIME,
              DROPOFF_DATETIME,
              DATE(PICKUP_DATETIME) AS PICKUP_DATE,
              HOUR(PICKUP_DATETIME) AS PICKUP_HOUR,
              DATE(DROPOFF_DATETIME) AS DROPOFF_DATE,
              HOUR(DROPOFF_DATETIME) AS DROPOFF_HOUR,
              DAYOFWEEK(PICKUP_DATETIME) AS DAY_OF_WEEK,
              MONTH(PICKUP_DATETIME) AS MONTH,
              YEAR(PICKUP_DATETIME) AS YEAR,

              PU_LOCATION_ID, PU_ZONE, PU_BOROUGH, PU_SERVICE_ZONE,
              DO_LOCATION_ID, DO_ZONE, DO_BOROUGH, DO_SERVICE_ZONE,

              SERVICE_TYPE, VENDOR_ID, VENDOR_NAME, RATE_CODE_ID, RATE_CODE_DESC,
              PAYMENT_TYPE, PAYMENT_TYPE_DESC, TRIP_TYPE, TRIP_TYPE_DESC,

              PASSENGER_COUNT, TRIP_DISTANCE, STORE_AND_FWD_FLAG,

              FARE_AMOUNT, EXTRA, MTA_TAX, TIP_AMOUNT, TOLLS_AMOUNT,
              IMPROVEMENT_SURCHARGE, CONGESTION_SURCHARGE, AIRPORT_FEE,
              CBD_CONGESTION_FEE, EHAIL_FEE, TOTAL_AMOUNT,

              ROUND(DATEDIFF(SECOND, PICKUP_DATETIME, DROPOFF_DATETIME) / 60.0, 2) AS TRIP_DURATION_MIN,
              CASE 
                WHEN TRIP_DISTANCE > 0 
                 AND DATEDIFF(SECOND, PICKUP_DATETIME, DROPOFF_DATETIME) > 0
                THEN ROUND(TRIP_DISTANCE / (DATEDIFF(SECOND, PICKUP_DATETIME, DROPOFF_DATETIME) / 3600.0), 2)
                ELSE NULL
              END AS AVG_SPEED_MPH,
              CASE 
                WHEN FARE_AMOUNT > 0 AND TIP_AMOUNT IS NOT NULL
                THEN ROUND((TIP_AMOUNT / FARE_AMOUNT) * 100, 2)
                ELSE 0.0
              END AS TIP_PCT,

              RUN_ID, INGESTED_AT_UTC, SOURCE_YEAR, SOURCE_MONTH
            FROM {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA_RAW}.UNIFIED_TRIPS
            WHERE YEAR(PICKUP_DATETIME) = {year} AND MONTH(PICKUP_DATETIME) = {month}
            QUALIFY ROW_NUMBER() OVER (
              PARTITION BY PICKUP_DATETIME, DROPOFF_DATETIME, PU_LOCATION_ID, DO_LOCATION_ID, SERVICE_TYPE
              ORDER BY INGESTED_AT_UTC DESC
            ) = 1
            """
            c.execute(insert_sql)
            inserted = c.rowcount or 0
            print(f"Filas insertadas: {inserted:,}")

            c.execute("COMMIT")
            print("Reingesta completada correctamente.")
        except Exception as e:
            c.execute("ROLLBACK")
            print(f"Error, rollback ejecutado: {e}")
            raise

        after_sql = f"""
        SELECT COUNT(*) 
        FROM {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA_ANALYTICS}.OBT_TRIPS
        WHERE YEAR(PICKUP_DATETIME) = {year} AND MONTH(PICKUP_DATETIME) = {month}
        """
        c.execute(after_sql)
        after_count = c.fetchone()[0]
        diff = after_count - before_count
        print(f"Registros después de reingesta: {after_count:,} (Δ {diff:+,})")


        dup_sql = f"""
        WITH g AS (
          SELECT PICKUP_DATETIME, DROPOFF_DATETIME, PU_LOCATION_ID, DO_LOCATION_ID, SERVICE_TYPE,
                 COUNT(*) AS cnt
          FROM {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA_ANALYTICS}.OBT_TRIPS
          WHERE YEAR(PICKUP_DATETIME) = {year} AND MONTH(PICKUP_DATETIME) = {month}
          GROUP BY 1,2,3,4,5
        )
        SELECT COUNT(*) FROM g WHERE cnt > 1
        """
        c.execute(dup_sql)
        dup_count = c.fetchone()[0]
        status = "Sin duplicados" if dup_count == 0 else f" {dup_count} duplicados detectados"
        print(f"Verificación duplicados: {status}")

        print(f"{'='*90}\n")


In [34]:
def verify_month(conn, year: int, month: int):
    print(f"\nVerificación del {year}-{month:02d}")

    # Duplicados por clave
    dup_sql = f"""
    with g as (
      select
        PICKUP_DATETIME, DROPOFF_DATETIME, PU_LOCATION_ID, DO_LOCATION_ID, SERVICE_TYPE,
        count(*) as cnt
      from {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA_ANALYTICS}.OBT_TRIPS
      where year(PICKUP_DATETIME) = {year} and month(PICKUP_DATETIME) = {month}
      group by 1,2,3,4,5
    )
    select count(*) from g where cnt > 1;
    """
    with conn.cursor() as c:
        c.execute(dup_sql)
        dups = c.fetchone()[0]
        print(f"  Duplicados por clave: {'✅ 0' if dups == 0 else f'⚠️ {dups} grupos con duplicados'}")

    # Conteo total del mes
    cnt_sql = f"""
    select count(*) 
    from {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA_ANALYTICS}.OBT_TRIPS
    where year(PICKUP_DATETIME) = {year} and month(PICKUP_DATETIME) = {month};
    """
    with conn.cursor() as c:
        c.execute(cnt_sql)
        total = c.fetchone()[0]
        print(f"  Total registros del mes: {total:,}")

In [35]:
print(f"\n{'='*80}")
print(f"VERIFICACIÓN DE CALIDAD")
print(f"{'='*80}")

# Total registros
cur.execute(f"SELECT COUNT(*) FROM {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA_ANALYTICS}.OBT_TRIPS")
total = cur.fetchone()[0]
print(f"\nTotal registros en OBT_TRIPS: {total:,}")

# Total columnas
cur.execute(f"""
    SELECT COUNT(*) 
    FROM {SNOWFLAKE_DATABASE}.INFORMATION_SCHEMA.COLUMNS 
    WHERE TABLE_SCHEMA = '{SNOWFLAKE_SCHEMA_ANALYTICS}' 
      AND TABLE_NAME = 'OBT_TRIPS'
""")
total_cols = cur.fetchone()[0]
print(f"Total columnas: {total_cols}")

# Por servicio
print("\nDistribución por servicio:")
cur.execute(f"""
    SELECT SERVICE_TYPE, COUNT(*) as cnt
    FROM {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA_ANALYTICS}.OBT_TRIPS
    GROUP BY SERVICE_TYPE
    ORDER BY SERVICE_TYPE
""")
for row in cur.fetchall():
    print(f"  {row[0]:10} {row[1]:>15,} registros")

# Por año
print("\nDistribución por año:")
cur.execute(f"""
    SELECT YEAR, COUNT(*) as cnt
    FROM {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA_ANALYTICS}.OBT_TRIPS
    GROUP BY YEAR
    ORDER BY YEAR
""")
for row in cur.fetchall():
    print(f"  {row[0]:10} {row[1]:>15,} registros")

# Nulos en columnas críticas
print("\nVerificando nulos en columnas críticas:")
critical_cols = ["PU_LOCATION_ID", "DO_LOCATION_ID", "TRIP_DISTANCE", "TOTAL_AMOUNT"]
for col_name in critical_cols:
    cur.execute(f"""
        SELECT COUNT(*) 
        FROM {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA_ANALYTICS}.OBT_TRIPS
        WHERE {col_name} IS NULL
    """)
    null_count = cur.fetchone()[0]
    status = "✅" if null_count == 0 else "⚠️"
    print(f"  {status} {col_name:20} {null_count:>10,} nulos")

# Verificar derivadas
print("\nVerificando columnas derivadas:")
cur.execute(f"""
    SELECT 
        COUNT(*) as total,
        COUNT(TRIP_DURATION_MIN) as with_duration,
        COUNT(AVG_SPEED_MPH) as with_speed,
        AVG(TIP_PCT) as avg_tip_pct
    FROM {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA_ANALYTICS}.OBT_TRIPS
""")
row = cur.fetchone()
print(f"  Total registros: {row[0]:,}")
print(f"  Con duración: {row[1]:,} ({row[1]/row[0]*100:.1f}%)")
print(f"  Con velocidad: {row[2]:,} ({row[2]/row[0]*100:.1f}%)")
print(f"  Tip promedio: {row[3]:.2f}%")


VERIFICACIÓN DE CALIDAD

Total registros en OBT_TRIPS: 852,374,316
Total columnas: 47

Distribución por servicio:
  green           68,045,440 registros
  yellow         784,328,876 registros

Distribución por año:
        2001              27 registros
        2002             498 registros
        2003              50 registros
        2004               1 registros
        2007               1 registros
        2008             884 registros
        2009           1,617 registros
        2010             349 registros
        2011               4 registros
        2012               4 registros
        2014               1 registros
        2015     165,272,997 registros
        2016     147,517,346 registros
        2017     125,237,292 registros
        2018     111,769,838 registros
        2019      90,898,123 registros
        2020      26,383,432 registros
        2021      31,972,712 registros
        2022      40,496,016 registros
        2023      39,097,193 registros
    

In [12]:
# Muestra de datos
print("\n Muestra de OBT_TRIPS (primeras 5 filas):")
cur.execute(f"""
    SELECT 
        PICKUP_DATETIME,
        SERVICE_TYPE,
        PU_BOROUGH,
        DO_BOROUGH,
        TRIP_DISTANCE,
        TRIP_DURATION_MIN,
        AVG_SPEED_MPH,
        TOTAL_AMOUNT,
        TIP_PCT
    FROM {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA_ANALYTICS}.OBT_TRIPS
    LIMIT 5
""")

for row in cur.fetchall():
    print(f"  {row}")


📋 Muestra de OBT_TRIPS (primeras 5 filas):
  (datetime.datetime(2015, 1, 29, 8, 3, 26), 'yellow', 'Brooklyn', 'Brooklyn', 1.1, Decimal('7.08'), 9.32, 7.3, 0.0)
  (datetime.datetime(2015, 1, 29, 8, 33, 12), 'yellow', 'Manhattan', 'Manhattan', 2.68, Decimal('24.57'), 6.55, 20.76, 20.97)
  (datetime.datetime(2015, 1, 29, 8, 15, 12), 'yellow', 'Queens', 'Manhattan', 12.1, Decimal('38.95'), 18.64, 58.67, 34.72)
  (datetime.datetime(2015, 1, 29, 8, 16, 44), 'yellow', 'Manhattan', 'Manhattan', 1.4, Decimal('11.50'), 7.3, 9.8, 0.0)
  (datetime.datetime(2015, 1, 29, 8, 47, 43), 'yellow', 'Manhattan', 'Manhattan', 1.3, Decimal('9.02'), 8.65, 9.95, 22.0)


In [36]:
print(f"\n{'='*80}")
print("EJEMPLO: Reingestar idempotente un mes")
print(f"{'='*80}")
Y, M = 2017, 9

reingest_month_delete_insert(conn, Y, M)
print("Listo.")

verify_month(conn, Y, M)

# %% [cleanup]
cur.close()
conn.close()
print("\nConexión cerrada.")


EJEMPLO: Reingestar idempotente un mes

 Reingesta mensual 2024-09
Registros previos en 2024-09: 3,629,090
Filas eliminadas: 3,629,090
⬆Filas insertadas: 3,629,090
Reingesta completada correctamente.
Registros después de reingesta: 3,629,090 (Δ +0)
Verificación duplicados: Sin duplicados

Listo.

Verificación del 2024-09
  Duplicados por clave: ✅ 0
  Total registros del mes: 3,629,090

Conexión cerrada.
